In [1]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import JSONLoader
from langchain.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings

load_dotenv()

file_path = "FAQ.json"

# Load the JSON File as Documents
loader = JSONLoader(file_path=file_path, jq_schema=".[]", text_content=False)
documents = loader.load()

# Initialize an Open-Source Embedding Model (using SentenceTransformers)
model_name = "all-MiniLM-L6-v2"  # Lightweight and free
# Create a HuggingFaceEmbeddings instance which internally uses SentenceTransformers
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Create a FAISS Vector Store from the Documents
faiss_db = FAISS.from_documents(documents, embeddings)

# (Optional) Persist the FAISS index locally for later use.
faiss_db.save_local("data/faiss_index")

c:\Users\TAMANG\Documents\GitHub\json-vector-rag-pipeline\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Example query text
query = "what is the product of the week?"

# Perform a similarity search
results = faiss_db.similarity_search(query, )

# Print out the results
for result in results:
    print(result)  # or result.metadata, depending on what you stored

page_content='{"question": "What is the product of the week?", "answer": "Our Upa Flowers are the product of the week."}' metadata={'source': 'C:\\Users\\TAMANG\\Documents\\GitHub\\json-vector-rag-pipeline\\FAQ.json', 'seq_num': 1}
page_content='{"question": "Do you offer subscription services?", "answer": "Yes, we offer weekly, bi-weekly, and monthly flower subscription services. You can choose the frequency, duration, and type of flowers you'd like to receive regularly."}' metadata={'source': 'C:\\Users\\TAMANG\\Documents\\GitHub\\json-vector-rag-pipeline\\FAQ.json', 'seq_num': 9}
page_content='{"question": "How far in advance should I order for special occasions like Valentine's Day?", "answer": "For major holidays like Valentine's Day, Mother's Day, or Christmas, we recommend placing your order at least 1-2 weeks in advance to ensure availability and timely delivery."}' metadata={'source': 'C:\\Users\\TAMANG\\Documents\\GitHub\\json-vector-rag-pipeline\\FAQ.json', 'seq_num': 15}
pa

### If the Computer is restarted then you can load local

In [3]:
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Load the persisted FAISS index with dangerous deserialization allowed
faiss_db = FAISS.load_local("data/faiss_index", embeddings, allow_dangerous_deserialization=True)

query = "what is the product of the week?"
results = faiss_db.similarity_search(query)

for result in results:
    print(result.page_content)

{"question": "What is the product of the week?", "answer": "Our Upa Flowers are the product of the week."}
{"question": "Do you offer subscription services?", "answer": "Yes, we offer weekly, bi-weekly, and monthly flower subscription services. You can choose the frequency, duration, and type of flowers you'd like to receive regularly."}
{"question": "How far in advance should I order for special occasions like Valentine's Day?", "answer": "For major holidays like Valentine's Day, Mother's Day, or Christmas, we recommend placing your order at least 1-2 weeks in advance to ensure availability and timely delivery."}
{"question": "What are your delivery options?", "answer": "We offer same-day delivery for orders placed before noon, next-day delivery, and scheduled delivery for future dates. Delivery options may vary based on your location and the availability of the flowers you've selected."}


### Code to add other json files (i.e., new data) into this existing vector db

In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import JSONLoader
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# Load environment variables
load_dotenv()

# Define paths for the new JSON files
new_files = [
    "privacy-policy.json",
    "common-questions.json"
]

# Initialize your embedding model (ensure it's the same as used before)
model_name = "all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Load the existing FAISS index from disk, allowing dangerous deserialization if needed
faiss_db = FAISS.load_local("data/faiss_index", embeddings, allow_dangerous_deserialization=True)

# Loop over each new JSON file, load documents and add them to the index
for file_path in new_files:
    # Load the JSON file as documents
    loader = JSONLoader(file_path=file_path, jq_schema=".[]", text_content=False)
    new_documents = loader.load()
    
    # Add the new documents to the existing FAISS index
    faiss_db.add_documents(new_documents)

# Optionally, persist the updated FAISS index
faiss_db.save_local("data/faiss_index")

In [ ]:
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Load the persisted FAISS index with dangerous deserialization allowed
faiss_db = FAISS.load_local("data/faiss_index", embeddings, allow_dangerous_deserialization=True)

query = "troubleshoot"
results = faiss_db.similarity_search(query)

for result in results:
    print(result.page_content)

---

In [4]:
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

def query_faiss(user_query, index_path="data/faiss_index", top_k=5):
    """
    Given a user query, this function loads the persisted FAISS index,
    performs a similarity search, and returns the top_k matching documents.
    
    Args:
        user_query (str): The input query from the user.
        index_path (str): The path where the FAISS index is saved.
        top_k (int): Number of similar documents to return.
        
    Returns:
        list: A list of the top_k matching document objects.
    """
    # Initialize the embedding model (should match the one used to build the index)
    model_name = "all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    
    # Load the persisted FAISS index (allow dangerous deserialization if needed)
    faiss_db = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)
    
    # Perform similarity search with the given query
    results = faiss_db.similarity_search(user_query, k=top_k)
    
    return results

In [5]:
# Example usage:
if __name__ == "__main__":
    query = input("Enter your query: ")
    similar_docs = query_faiss(query)
    
    print("\nTop matching documents:")
    for doc in similar_docs:
        print("-----")
        print(doc.page_content)  # or use doc.metadata if that's more appropriate


Top matching documents:
-----
{"question": "What types of flowers do you offer?", "answer": "We offer a wide variety of flowers including roses, lilies, tulips, sunflowers, orchids, and seasonal bouquets. Our selection changes regularly to ensure freshness and variety."}
-----
{"question": "What if the flowers I want are out of season?", "answer": "If certain flowers are out of season, we offer similar alternatives or can create custom arrangements using in-season blooms that match your desired style and color scheme."}
-----
{"question": "How do I ensure my flowers stay fresh?", "answer": "To keep your flowers fresh, change the water every 2-3 days, trim the stems at an angle, remove any leaves below the waterline, and keep the flowers away from direct sunlight and heat sources."}
-----
{"question": "How do I know which flowers are appropriate for different occasions?", "answer": "Our website provides guidance on flower meanings and appropriate choices for various occasions. You can 

---

In [6]:
import os
from dotenv import load_dotenv
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamlitCallbackHandler  # example callback for streaming output
from langchain.schema import HumanMessage, SystemMessage

# Load environment variables (make sure OPENAI_API_KEY is set)
load_dotenv()


def construct_rag_prompt(query, retrieved_docs):
    """
    Build a prompt that includes retrieved document content and the user's query.
    """
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    prompt = (
        "You are a helpful assistant that uses the following context to answer the question.\n\n"
        "Context:\n"
        f"{context}\n\n"
        "Question:\n"
        f"{query}\n\n"
        "Note: If the 'Question' the user asks is not present in the 'Context', then you will respond with 'Unfortunately we do not have answer for that."
    )
    return prompt

In [8]:
from openai import OpenAI

# Get user's query
user_query = input("Enter your query: ")

# Step 1: Retrieve similar documents using the FAISS index
retrieved_docs = query_faiss(user_query, index_path="data/faiss_index", top_k=3)

# Step 2: Construct the RAG prompt
prompt = construct_rag_prompt(user_query, retrieved_docs)
print("\n\n--- RAG Prompt ---\n")
print(prompt)
print("\n\n--- Streaming Response ---\n")

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

stream = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that answers questions using provided context."},
        {"role": "user", "content": prompt}
    ],
    stream=True,  # Enable streaming mode
    temperature=0.7,
    max_tokens=1000,
)

answer = ""

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")



--- RAG Prompt ---

You are a helpful assistant that uses the following context to answer the question.

Context:
{"question": "What is the product of the week?", "answer": "Our Upa Flowers are the product of the week."}

{"question": "What are your delivery options?", "answer": "We offer same-day delivery for orders placed before noon, next-day delivery, and scheduled delivery for future dates. Delivery options may vary based on your location and the availability of the flowers you've selected."}

{"question": "Do you offer subscription services?", "answer": "Yes, we offer weekly, bi-weekly, and monthly flower subscription services. You can choose the frequency, duration, and type of flowers you'd like to receive regularly."}

Question:
what is the product of the day?

Note: If the 'Question' the user asks is not present in the 'Context', then you will respond with 'Unfortunately we do not have answer for that.


--- Streaming Response ---

Unfortunately we do not have answer for th